<a href="https://colab.research.google.com/github/ajcoder13/CapstoneSA/blob/main/CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [226]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [227]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [228]:
from google.colab import drive
drive.mount('/content/Drive')
df = pd.read_csv('/content/Drive/My Drive/Colab Notebooks/dataset.csv')
df


Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [229]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [230]:
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].astype(object)
#changed datatype from float64 to convert these into numeric for model implementation

In [231]:
traffic_map = {
    'low': 0,
    'average': 1,
    'high': 2
}
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].str.lower().str.strip()
df['TrafficConditionNumeric'] = df['TrafficConditionNearby'].map(traffic_map)
print(df[['TrafficConditionNearby', 'TrafficConditionNumeric']].head())

  TrafficConditionNearby  TrafficConditionNumeric
0                    low                        0
1                    low                        0
2                    low                        0
3                    low                        0
4                    low                        0


In [232]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,TrafficConditionNumeric
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0


In [233]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [234]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location


In [235]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [236]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID                       18368 non-null  int64         
 1   SystemCodeNumber         18368 non-null  object        
 2   Capacity                 18368 non-null  int64         
 3   Latitude                 18368 non-null  float64       
 4   Longitude                18368 non-null  float64       
 5   Occupancy                18368 non-null  int64         
 6   VehicleType              18368 non-null  object        
 7   TrafficConditionNearby   18368 non-null  object        
 8   QueueLength              18368 non-null  int64         
 9   IsSpecialDay             18368 non-null  int64         
 10  LastUpdatedDate          18368 non-null  object        
 11  LastUpdatedTime          18368 non-null  object        
 12  Timestamp                18368 n

# Step 2: Making a simple pricing function

In [238]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(seconds=10)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
    )
    .with_columns(
        # Compute the price using a simple dynamic pricing formula:
        #
        # Pricing Formula:
        #     price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_max - occ_min) / cap
        #
        # Intuition:
        # - The greater the difference between peak and low occupancy in a day,
        #   the more volatile the demand is, indicating potential scarcity.
        # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
        # - This fluctuation is added to the base price of 10 to set the final price.
        # - Example: If occ_max = 90, occ_min = 30, cap = 100
        #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6

        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)


# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [239]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [240]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

# Model 1: Baseline Linear Model

In [241]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
# Parameters
BASE_PRICE = 10.0
ALPHA = 0.4      # Occupancy weight
BETA = 0.06        # Queue weight
GAMMA = 0.5      # Traffic dampening
PRICE_MIN = 5.0
PRICE_MAX = 20.0

advanced_price_data = []

for lot_id in df["SystemCodeNumber"].unique():
    lot_df = df[df["SystemCodeNumber"] == lot_id].copy().reset_index(drop=True)
    prices = [BASE_PRICE]

    for i in range(1, len(lot_df)):
        prev_price = prices[-1]
        occ = lot_df.loc[i, "Occupancy"]
        cap = lot_df.loc[i, "Capacity"]
        queue = lot_df.loc[i, "QueueLength"]
        traffic = lot_df.loc[i, "TrafficConditionNumeric"]
        #previous price adds to last of array

        # Occupancy influence
        occ_ratio = occ / cap

        # Queue nonlinear score: log-scale dampens spikes
        queue_score = np.log1p(queue)


        # Advanced price calculation
        new_price = prev_price + ALPHA * occ_ratio
        new_price += BETA * queue_score #added new parameter
        new_price -= GAMMA * traffic #added new parameter
        # Clip to smooth range
        new_price = max(PRICE_MIN, min(PRICE_MAX, round(new_price, 2)))
        prices.append(new_price)

    lot_df["price"] = prices
    advanced_price_data.append(lot_df)

result_df = pd.concat(advanced_price_data)

# Linear plot
plots = []
for lot_id in result_df["SystemCodeNumber"].unique():
    data = result_df[result_df["SystemCodeNumber"] == lot_id]
    p = figure(title=f"Advanced Price - Lot {lot_id}", x_axis_type='datetime', width=400, height=250)
    p.line(data["Timestamp"], data["price"], line_width=2, color="grey")
    p.circle(data["Timestamp"], data["price"], size=5, color="#EA3769", alpha=0.5)
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    plots.append(p)

show(gridplot([plots[i:i+3] for i in range(0, len(plots), 3)]))

# Model 2: Demand-Based Price Function

In [242]:
BASE_PRICE = 10.0
ALPHA = 0.5   # Occupancy influence
BETA = 0.06    # Queue length impact
GAMMA = 0.2   # Traffic dampener
DELTA = 0.5   # Boost for special day
EPSILON = {"Car": 1.0, "Bike": 0.6, "Truck": 1.5}  # Vehicle weights
LAMBDA = 0.5  # Demand sensitivity

In [243]:
# Normalize & clean key features
df["TrafficConditionNearby"] = df["TrafficConditionNearby"].map({"low": 1, "average": 2, "high": 3})

df["VehicleTypeWeight"] = df["VehicleType"].map(EPSILON).fillna(1.0)
df["IsSpecialDay"] = df["IsSpecialDay"].astype(int)

In [244]:
# Avoid division by zero
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]
df["QueueLength"] = pd.to_numeric(df["QueueLength"], errors="coerce").fillna(0)
df["Traffic"] = pd.to_numeric(df["TrafficConditionNearby"], errors="coerce")

# Raw demand score
df["RawDemand"] = (
    ALPHA * df["OccupancyRate"] +
    BETA * df["QueueLength"] -
    GAMMA * df["Traffic"] +
    DELTA * df["IsSpecialDay"] +
    df["VehicleTypeWeight"]
)

# Normalize demand across all records
dmin, dmax = df["RawDemand"].min(), df["RawDemand"].max()
df["NormalizedDemand"] = (df["RawDemand"] - dmin) / (dmax - dmin)

In [245]:
df["Price"] = BASE_PRICE * (1 + LAMBDA * df["NormalizedDemand"])

# Bound prices between 0.5x and 2x base price
df["Price"] = df["Price"].clip(lower=BASE_PRICE * 0.5, upper=BASE_PRICE * 2)

In [246]:
#converted queue length to queue score using log
df["QueueScore"] = np.log1p(df["QueueLength"])
df["OccupancyScore"] = (df["Occupancy"] / df["Capacity"])**0.8

In [247]:
df["TrafficImpact"] = df["Traffic"] * df["IsSpecialDay"]

In [248]:
#smoothig function
df["SmoothedDemand"] = df["NormalizedDemand"].rolling(window=3, min_periods=1).mean()
df["Price"] = BASE_PRICE * (1 + LAMBDA * df["SmoothedDemand"])

In [249]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot

output_notebook()

plots = []

for lot_id in df["SystemCodeNumber"].unique():
    data = df[df["SystemCodeNumber"] == lot_id]
    p = figure(title=f"Model 2 Price - Lot {lot_id}", x_axis_type='datetime', width=400, height=250)
    p.line(data["Timestamp"], data["Price"], line_width=2, color="green")
    p.circle(data["Timestamp"], data["Price"], size=5, color="blue", alpha=0.5)
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    plots.append(p)

show(gridplot([plots[i:i+3] for i in range(0, len(plots), 3)]))


# MODEL 3 : Competitive Pricing Model

In [250]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # Distance in km

In [251]:
parking_lots = df[["SystemCodeNumber", "Latitude", "Longitude", "Price"]].drop_duplicates().to_dict("records")

In [252]:
for each_lot in parking_lots:
    nearby_lots = [lot for lot in parking_lots if lot["SystemCodeNumber"] != each_lot["SystemCodeNumber"]]

    occupancy = each_lot.get("Occupancy", 0)
    capacity = each_lot.get("Capacity", 1)  # Avoid divide by zero
    current_price = each_lot.get("Price", BASE_PRICE)

    nearby_prices = [lot["Price"] for lot in nearby_lots]

    COMPETITION_DISCOUNT = current_price - min(nearby_prices)
    suggest_reroute = False  # Default value
    if occupancy == capacity and any(p < current_price for p in nearby_prices):
        suggest_reroute = True
        adjusted_price = max(BASE_PRICE, current_price - COMPETITION_DISCOUNT)
        print(f"Lot {each_lot['SystemCodeNumber']} suggests rerouting or reduced price: ${adjusted_price}")

In [253]:
if occupancy == capacity and any(nearby_price < current_price for nearby_price in nearby_prices):
    suggest_reroute = True
    price = max(BASE_PRICE, current_price - COMPETITION_DISCOUNT)

In [254]:
MAX_PRICE=60.0
if all(nearby_price > current_price for nearby_price in nearby_prices):
    adjusted_price = min(min(nearby_prices), MAX_PRICE)

In [256]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot

output_notebook()

plots = []
for lot_id in df["SystemCodeNumber"].unique():
    lot_df = df[df["SystemCodeNumber"] == lot_id]
    competitor_df = df[(df["SystemCodeNumber"] != lot_id) & (df["Timestamp"].isin(lot_df["Timestamp"]))]

    p = figure(title=f"Lot {lot_id} vs Competitors", x_axis_type='datetime', width=400, height=250)

    # lot's price
    p.line(lot_df["Timestamp"], lot_df["Price"], line_width=2, color="blue", legend_label="Your Price")

    # Competitor price average over matching timestamps
    if not competitor_df.empty:
        avg_competitor_price = competitor_df.groupby("Timestamp")["Price"].mean()
        p.line(avg_competitor_price.index, avg_competitor_price.values,
               line_width=2, color="red", legend_label="Competitor Avg")

    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    p.legend.location = "top_right"
    plots.append(p)
show(gridplot([plots[i:i+3] for i in range(0, len(plots), 3)]))